In [1]:
import gc
import time
import numpy as np
import pandas as pd
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import lightgbm as lgb
from lightgbm import plot_importance
import matplotlib.pyplot as plt
import sys
import os

In [2]:
path='E:\\kaggle\\Avito Demand Prediction Challenge\\features\\dataset\\'
train=pd.read_csv(path+'trainx.csv',header=0)
train.drop(['item_id','user_id','title','description'], axis=1, inplace=True )
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

skf = KFold(n_splits = 25, shuffle=True,random_state=123456789)
count=0
#seed=np.random.randint(0,10)
seed=13
for train_idx, test_idx in skf.split(train, train.deal_probability):
    if count==seed:
        train_id=train_idx
        test_id=test_idx
    count=count+1
    
train_x, valid_x = train.ix[train_id], train.ix[test_id]
del train
gc.collect()

train_y = train_x['deal_probability']
train_x.drop(['deal_probability'], axis=1, inplace=True )
print('Size of the training set is {}'.format(train_x.shape))

valid_y = valid_x['deal_probability']
valid_x.drop(['deal_probability'], axis=1, inplace=True )
print('Size of the validation set is {}'.format(valid_x.shape))

H:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Size of the training set is (1443287, 93)
Size of the validation set is (60137, 93)


In [ ]:
predictors = list(train_x.columns)
categorical = ['region','city','parent_category_name','category_name', 
              'user_type','activation_date', 'param_1', 'param123','image_top_1',
              'cluster_5', 'cluster_10','cluster_20','item_seq_number']

lgb_train = lgb.Dataset(train_x, label=train_y, feature_name=predictors, categorical_feature=categorical)
lgb_valid = lgb.Dataset(valid_x, label=valid_y, feature_name=predictors, categorical_feature=categorical)

del train_x, train_y, valid_x, valid_y 
gc.collect()

lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'xentropy',
        'metric':'rmse',
        'learning_rate': 0.001,
        #'is_unbalance': 'true',  #because training data is unbalance (replaced with scale_pos_weight)
        'num_leaves': 1024,  # we should let it be smaller than 2^(max_depth)
        'max_depth': 40, #-1 # -1 means no limit
        #'min_data_in_leaf': 10,
        'max_bin': 255,  # Number of bucketed bin for feature values
        'feature_fraction': 0.5, # 0.5
        'bagging_fraction': 0.75, # 0.75
        'bagging_freq': 1, #2
        'min_split_gain': 0.0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'lambda_l1': 0.0,  # L1 regularization term on weights
        'lambda_l2': 0.01,  # L2 regularization term on weights
        'nthread': 4,
        'verbose': 0,
        'is_training_metric': True
        #'scale_pos_weight':99
    }

model_lgb = lgb.train(lgb_params, lgb_train, valid_sets=[lgb_train,lgb_valid], valid_names=['train','valid'],
                      num_boost_round=50000, early_stopping_rounds=200, verbose_eval=50)
#model_lgb.save_model('model.txt')

H:\anaconda\envs\tensorflow\lib\site-packages\lightgbm\basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
H:\anaconda\envs\tensorflow\lib\site-packages\lightgbm\basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[50]	train's rmse: 0.256989	valid's rmse: 0.255614
[100]	train's rmse: 0.254066	valid's rmse: 0.252947
[150]	train's rmse: 0.251319	valid's rmse: 0.250468
[200]	train's rmse: 0.248762	valid's rmse: 0.248177
[250]	train's rmse: 0.24636	valid's rmse: 0.246047
[300]	train's rmse: 0.244107	valid's rmse: 0.244072
[350]	train's rmse: 0.24201	valid's rmse: 0.242254
[400]	train's rmse: 0.240033	valid's rmse: 0.240555
[450]	train's rmse: 0.238191	valid's rmse: 0.238992
[500]	train's rmse: 0.236456	valid's rmse: 0.237543
[550]	train's rmse: 0.234823	valid's rmse: 0.236193
[600]	train's rmse: 0.233304	valid's rmse: 0.234949
[650]	train's rmse: 0.231888	valid's rmse: 0.233821
[700]	train's rmse: 0.230558	valid's rmse: 0.232767
[750]	train's rmse: 0.22931	valid's rmse: 0.231796
[800]	train's rmse: 0.228112	valid's rmse: 0.230878
[850]	train's rmse: 0.226997	valid's rmse: 0.230037
[900]	train's rmse: 0.225941	valid's rmse: 0.229256
[950]

[7800]	train's rmse: 0.180925	valid's rmse: 0.216048
[7850]	train's rmse: 0.180725	valid's rmse: 0.21604
[7900]	train's rmse: 0.180524	valid's rmse: 0.216031
[7950]	train's rmse: 0.180329	valid's rmse: 0.216021
[8000]	train's rmse: 0.180129	valid's rmse: 0.216014
[8050]	train's rmse: 0.179936	valid's rmse: 0.216007
[8100]	train's rmse: 0.179742	valid's rmse: 0.215999
[8150]	train's rmse: 0.179547	valid's rmse: 0.21599
[8200]	train's rmse: 0.179349	valid's rmse: 0.215982
[8250]	train's rmse: 0.179155	valid's rmse: 0.215976
[8300]	train's rmse: 0.178969	valid's rmse: 0.215967
[8350]	train's rmse: 0.178785	valid's rmse: 0.215958
[8400]	train's rmse: 0.178601	valid's rmse: 0.215951
[8450]	train's rmse: 0.178414	valid's rmse: 0.215945
[8500]	train's rmse: 0.178223	valid's rmse: 0.215937
[8550]	train's rmse: 0.178034	valid's rmse: 0.215931
[8600]	train's rmse: 0.177846	valid's rmse: 0.215924
[8650]	train's rmse: 0.177658	valid's rmse: 0.215918
[8700]	train's rmse: 0.177469	valid's rmse: 0.21

[15500]	train's rmse: 0.155978	valid's rmse: 0.215548
[15550]	train's rmse: 0.155841	valid's rmse: 0.215548
[15600]	train's rmse: 0.155702	valid's rmse: 0.215548
[15650]	train's rmse: 0.155565	valid's rmse: 0.215549
[15700]	train's rmse: 0.155428	valid's rmse: 0.215549
[15750]	train's rmse: 0.155294	valid's rmse: 0.215549
Early stopping, best iteration is:
[15559]	train's rmse: 0.155815	valid's rmse: 0.215547


In [4]:
model_lgb.save_model('modelxsssss.txt')
#bst = lgb.Booster(model_file='mode.txt')

In [5]:
path='E:\\kaggle\\Avito Demand Prediction Challenge\\features\\dataset\\'
test=pd.read_csv(path+'testx.csv',header=0)
item=test['item_id']
test.drop(['item_id','user_id','title','description'], axis=1, inplace=True )
predictors = list(test.columns)
categorical = ['region','city','parent_category_name','category_name', 
              'user_type','activation_date', 'param_1', 'param123','image_top_1',
              'cluster_5', 'cluster_10','cluster_20','item_seq_number']
lgb_test = lgb.Dataset(test, feature_name=predictors, categorical_feature=categorical)

y_preds=model_lgb.predict(test, num_iteration=model_lgb.best_iteration)

result = {'item_id':item, 'deal_probability': y_preds}
result = pd.DataFrame(data=result)
result=result[['item_id','deal_probability']]
result.to_csv('resultsssss.001.csv', index=False) 

In [6]:
y_preds=model_lgb.predict(test, num_iteration=model_lgb.best_iteration)

In [7]:
result = {'item_id':item, 'deal_probability': y_preds}
result = pd.DataFrame(data=result)
result=result[['item_id','deal_probability']]
result.to_csv('result0.001.csv', index=False) 

In [8]:
y_preds

array([0.05546752, 0.13976673, 0.19719934, ..., 0.04052349, 0.35874685,
       0.0004964 ])